In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataset import random_split

import matplotlib as mpl
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
csv_path = "final.csv"
image_path = "./images"
batch_size = 400
epochs = 70
learning_rate = 1e-3

In [3]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, csv_path, image_path, csv_transform=None, image_transform=transforms.Compose([transforms.ToTensor()])):
        super(Dataset).__init__()
        csv = pd.read_csv(csv_path)
        csv_np = csv.to_numpy()
        self.images = []
        for Id in csv['Id']:
            Id = str(Id)
            image = Image.open(image_path+'/'+Id+'.jpg')
            self.images.append(image_transform(image))
            image.close()
        
        for i in [1, 3, 4, 5, 6, 7, 8]:
            wordset = {word: idx for idx, word in enumerate(np.unique(csv_np[:,i]))}
            for row in range(len(csv_np)):
                csv_np[row][i] = wordset[csv_np[row][i]]
        self.ints = torch.from_numpy(np.array(csv_np[:,[1,3,4,5,6,7,8]], dtype="int"))
        self.floats = torch.from_numpy(np.array(csv_np[:,[2,9,10,11]], dtype="float")).float()
        if csv_transform is not None:
            self.floats = torch.tensor(csv_transform(self.floats), dtype = torch.float)
        self.target = torch.from_numpy(np.array(csv_np[:,[12]], dtype="float")).float()
    
    
    def __getitem__(self,idx):
        return self.images[idx], self.ints[idx],self.floats[idx], self.target[idx]
    
    
    def __len__(self):
        return len(self.ints)

In [4]:
data_length = len(pd.read_csv(csv_path))
train_length = int(data_length * 0.6)
test_length = int(data_length * 0.2)
val_length = data_length - train_length - test_length

csv_transform = None
image_transform = transforms.Compose([transforms.ToTensor()])

train_dataset = Dataset(csv_path, image_path, csv_transform, image_transform)
train_dataset, test_dataset = random_split(train_dataset, [train_length, test_length+val_length])
test_dataset, val_dataset = random_split(test_dataset, [test_length, val_length])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle = True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle = True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle = True)

In [5]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.resnet18 = torchvision.models.resnet18(pretrained=False)
        self.resnet18.conv1 = nn.Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.resnet18.fc = nn.Sequential(nn.Linear(512, 4096), nn.ReLU())
        self.resenet18 = self.resnet18.to(device)
        
        self.emb1 = torch.nn.Embedding(8, 20)
        self.emb2 = torch.nn.Embedding(11, 20)
        self.emb3 = torch.nn.Embedding(20, 20)
        self.emb4 = torch.nn.Embedding(4, 20)
        self.emb5 = torch.nn.Embedding(4, 20)
        self.emb6 = torch.nn.Embedding(4, 20)
        self.emb7 = torch.nn.Embedding(7, 20)
        self.act = nn.ReLU()
        self.fc = nn.Linear(4, 80)
        self.csvfc1 = nn.Linear(220, 8192)
        self.csvfc2 = nn.Linear(8192, 8192)
        self.csvfc3 = nn.Linear(8192, 4096)
        
        self.fc1 = nn.Linear(8192, 8192)
        self.fc2 = nn.Linear(8192, 4096)
        self.fc3 = nn.Linear(4096, 2048)
        self.fc4 = nn.Linear(2048, 1)
        self.dropout = nn.Dropout()
    
    def forward(self, image, x, y):
        image = self.resnet18(image)
        
        x1 = self.emb1(x[:,0])
        x2 = self.emb2(x[:,1])
        x3 = self.emb3(x[:,2])
        x4 = self.emb4(x[:,3])
        x5 = self.emb5(x[:,4])
        x6 = self.emb6(x[:,5])
        x7 = self.emb7(x[:,6])
        y = self.fc(y)
        x = torch.cat((x1, x2, x3, x4, x5, x6, x7, y), dim=1)
        
        x = self.dropout(self.act(self.csvfc1(x)))
        x = self.dropout(self.act(self.csvfc2(x)))
        x = self.dropout(self.act(self.csvfc3(x)))
        x = torch.cat((x, image), dim=1)
        
        x = self.act(self.fc1(x))
        x = self.act(self.fc2(x))
        x = self.act(self.fc3(x))
        return self.fc4(x)

In [6]:
model = Net().to(device)

criterion = nn.MSELoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [7]:
for epoch in range(epochs):
    model.train()
    criterion.train()
    
    avg_loss = 0

    for Image, X1, X2, Y in train_loader:
        Image = Image.to(device)
        X1 = X1.to(device)
        X2 = X2.to(device)
        Y = Y.to(device)

        model.zero_grad()  # why we use zero_grad?
        prediction = model(Image, X1, X2)
        loss = torch.sqrt(criterion(prediction, Y)).to(device)
        loss.backward()
        optimizer.step()
        avg_loss += loss / len(train_loader)
    print(f'[Epoch: {epoch+1:>2}] Average loss: {avg_loss:.4f}, ', end='')
    
    model.eval()
    criterion.eval()
    with torch.no_grad():
        val_avg_loss = 0.
        for Image_val, X1_val, X2_val, Y_val in val_loader:
            Image_val = Image_val.to(device)
            X1_val = X1_val.to(device)
            X2_val = X2_val.to(device)
            Y_val = Y_val.to(device)
            val_prediction = model(Image_val, X1_val, X2_val)
            val_loss = torch.sqrt(criterion(val_prediction, Y_val)).to(device)
            val_avg_loss += val_loss / len(val_loader)
        
        print(f"val_loss: {val_avg_loss:.4f}")

[Epoch:  1] Average loss: 9250.9082, val_loss: 4809.6963
[Epoch:  2] Average loss: 4477.8555, val_loss: 5128.0391
[Epoch:  3] Average loss: 3833.0349, val_loss: 4807.1484
[Epoch:  4] Average loss: 3467.8557, val_loss: 5305.4473
[Epoch:  5] Average loss: 3586.7563, val_loss: 3557.1631
[Epoch:  6] Average loss: 4161.3203, val_loss: 3091.5693
[Epoch:  7] Average loss: 3370.6348, val_loss: 28112.0469
[Epoch:  8] Average loss: 3174.0352, val_loss: 73426.5938
[Epoch:  9] Average loss: 2884.8325, val_loss: 114073.8047
[Epoch: 10] Average loss: 2912.8489, val_loss: 67336.7109
[Epoch: 11] Average loss: 2615.5059, val_loss: 6740.7998
[Epoch: 12] Average loss: 2957.8135, val_loss: 2092.3462
[Epoch: 13] Average loss: 2160.2375, val_loss: 2752.5725
[Epoch: 14] Average loss: 2426.1841, val_loss: 18739.5293
[Epoch: 15] Average loss: 2492.4585, val_loss: 2576.2170
[Epoch: 16] Average loss: 2620.5085, val_loss: 84148.4141
[Epoch: 17] Average loss: 2746.3887, val_loss: 2078.5078
[Epoch: 18] Average loss

In [8]:
model.eval()
criterion.eval()
ss_tot = 0
ss_res = 0
with torch.no_grad():
    for Image_test, X1_test, X2_test, Y_test in test_loader:
        Image_test =Image_test.to(device)
        X1_test = X1_test.to(device)
        X2_test = X2_test.to(device)
        Y_test = Y_test.to(device)
        prediction = model(Image_test, X1_test, X2_test)
        prices_mean = torch.mean(Y_test)
        ss_tot += torch.sum((Y_test - prices_mean) ** 2)
        ss_res += torch.sum((Y_test - prediction) ** 2)
    
    accuracy = 1 - ss_tot/ss_res
    print(f"Accuracy: {accuracy*100:.2f}%")

Accuracy: 81.67%
